# This Notebook Is for Exploring Importing Data from IEX into Apache Cassandra
## Future works will be converted into python scripts which will be more easily 'dockerized' for managing requirements and run environments
### Although Cassandra's advantages are more in enterprise-level write performance, I think its widespread use and compatibility with DFS will make it a better option than Mongo for any crazy timelines where this turns into anything

In [115]:
# Import Requirements
import requests
import csv
from bs4 import BeautifulSoup
from cassandra.cluster import Cluster

In [70]:
#Establish a Cluster object that will connect to the cassandra instances
cluster = Cluster()

In [87]:
#Use requests session to pull Wikipedia's S&P500 list page, this will hopefully not change too much or else this might break
session = requests.Session()
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = session.get(url)
#Pull the content (html) of the HTTP response object into BSoup and use the html parser
soup = BeautifulSoup(response.content,'html.parser')

In [110]:
#Utilize the constituents id to locate the relavant HTML info
const_table = soup.find(id='constituents')
#More parsing...
table_row = const_table.find_all('tr')[1:] #Use of list indexing removes the table header which is also a tr
#Create an empty list to receive the tickers for the S&P500 companies
sp500 = []
#Loop over BSoup result set, each 'row' being a BSoup object that can be further parsed into table data
for row in table_row:
    columns = row.find_all('td')
    #Based on the layout of the table, the second column will contain the symbols
    symbol_col = columns[1]
    sp500.append(symbol_col.text)
#Confirm the length of the list as 505
print(len(sp500))

505


In [139]:
#Write list to a csv file for durability
with open ('./sp500.csv','w',newline='') as csvfile:
    csvwriter = csv.writer(csvfile,delimiter=',')
    for ticker in sp500:
        csvwriter.writerow([ticker])

In [142]:
#Check the csv file
with open('./sp500.csv','r',newline='') as csvfile:
    csvreader = csv.reader(csvfile)
    counter = 0
    for row in csvreader:
        counter += 1
        print(row, counter)

['MMM'] 1
['ABT'] 2
['ABBV'] 3
['ABMD'] 4
['ACN'] 5
['ATVI'] 6
['ADBE'] 7
['AMD'] 8
['AAP'] 9
['AES'] 10
['AMG'] 11
['AFL'] 12
['A'] 13
['APD'] 14
['AKAM'] 15
['ALK'] 16
['ALB'] 17
['ARE'] 18
['ALXN'] 19
['ALGN'] 20
['ALLE'] 21
['AGN'] 22
['ADS'] 23
['LNT'] 24
['ALL'] 25
['GOOGL'] 26
['GOOG'] 27
['MO'] 28
['AMZN'] 29
['AEE'] 30
['AAL'] 31
['AEP'] 32
['AXP'] 33
['AIG'] 34
['AMT'] 35
['AWK'] 36
['AMP'] 37
['ABC'] 38
['AME'] 39
['AMGN'] 40
['APH'] 41
['APC'] 42
['ADI'] 43
['ANSS'] 44
['ANTM'] 45
['AON'] 46
['AOS'] 47
['APA'] 48
['AIV'] 49
['AAPL'] 50
['AMAT'] 51
['APTV'] 52
['ADM'] 53
['ARNC'] 54
['ANET'] 55
['AJG'] 56
['AIZ'] 57
['T'] 58
['ADSK'] 59
['ADP'] 60
['AZO'] 61
['AVB'] 62
['AVY'] 63
['BHGE'] 64
['BLL'] 65
['BAC'] 66
['BK'] 67
['BAX'] 68
['BBT'] 69
['BDX'] 70
['BRK-B'] 71
['BBY'] 72
['BIIB'] 73
['BLK'] 74
['HRB'] 75
['BA'] 76
['BKNG'] 77
['BWA'] 78
['BXP'] 79
['BSX'] 80
['BHF'] 81
['BMY'] 82
['AVGO'] 83
['BR'] 84
['BF-B'] 85
['CHRW'] 86
['COG'] 87
['CDNS'] 88
['CPB'] 89
['COF'] 